# RQ1 - Eyetracking Fixation Metrics

## Import Libraries

In [3]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import utils.GenSnippetsLib as gsl
import json
import os

In [4]:
screen_resolution = (1920, 1080)

## Import Eyetracking Data

In [5]:
df_query = pd.read_csv("./data/filteredData/filtered_data.csv")
df_eyetracking_events = pd.DataFrame(columns=["Participant", "Algorithm", "Path"])
snippets = df_query["Algorithm"].unique()
participants = df_query["Participant"].unique()
for participant in participants:
    for snippet in snippets:
        path = f"./data/filteredData/Participant{str(participant).zfill(2)}/{snippet}_Code_eyetracking.csv"
        # check if path exists
        if os.path.exists(path):
            df_eyetracking_events.loc[len(df_eyetracking_events)] = [participant, snippet, path]
df_eyetracking_events

,Participant,Algorithm,Path
0,1,IsPrime,./data/filteredData/Participant01/IsPrime_Code...
1,1,SiebDesEratosthenes,./data/filteredData/Participant01/SiebDesErato...
2,1,IsAnagram,./data/filteredData/Participant01/IsAnagram_Co...
3,1,RemoveDoubleChar,./data/filteredData/Participant01/RemoveDouble...
4,1,BinToDecimal,./data/filteredData/Participant01/BinToDecimal...
...,...,...,...
1067,71,BogoSort,./data/filteredData/Participant71/BogoSort_Cod...
1068,71,ReverseQueue,./data/filteredData/Participant71/ReverseQueue...
1069,71,Ackerman,./data/filteredData/Participant71/Ackerman_Cod...
1070,71,RabbitTortoise,./data/filteredData/Participant71/RabbitTortoi...


In [6]:
df_filtered = pd.read_csv("./data/filteredData/filtered_data.csv")
Algorithms = df_filtered["Algorithm"].unique()
Participants = df_filtered[df_filtered["IsOutlier"] == False]["Participant"].unique()

In [7]:
def doBoxesCollide(a, b):
    a_x_center = a[0]
    a_y_center = a[1]
    a_width = a[2]
    a_height = a[3]
    b_x_center = b[0]
    b_y_center = b[1]
    b_width = b[2]
    b_height = b[3]
    return abs(a_x_center - b_x_center) * 2 < (a_width + b_width) and abs(a_y_center - b_y_center) * 2 < (a_height + b_height)

# Calculate Line based Metrics

In [8]:
# Get Bounding Boxes for Lines Of Code
df_lines = pd.DataFrame(columns=["Algorithm", "Line", "BoundingBox"])
for snippet in tqdm(Algorithms):
    aoi_token_generator = f"./../CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="./../CodeSnippets/fonts/ttf/")
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = None
    current_top = None
    current_right = None
    current_bottom = None
    current_line = 0
    for letter in aoi_list:
        if letter["letter"] == '\n':
            if current_left is not None:
                aoi_clustered.append((current_line, current_left, current_top, current_right, current_bottom))
            current_left = None
            current_top = None
            current_right = None
            current_bottom = None
            current_line += 1
            continue
        if current_left is None:
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]
        else:
            current_left = min(current_left, letter["BoundingBox"][0])
            current_top = min(current_top, letter["BoundingBox"][1])
            current_right = max(current_right, letter["BoundingBox"][2])
            current_bottom = max(current_bottom, letter["BoundingBox"][3])

    for token in aoi_clustered:
        df_lines.loc[len(df_lines)] = [snippet, token[0],
                                       (token[1] + width_offset,
                                        token[2] + height_offset,
                                        token[3] + width_offset,
                                        token[4] + height_offset)]
df_lines

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Line,BoundingBox
0,IsPrime,0,"(768, 467, 1152, 482)"
1,IsPrime,1,"(768, 486, 1112, 501)"
2,IsPrime,2,"(768, 505, 1049, 519)"
3,IsPrime,3,"(768, 526, 1008, 539)"
4,IsPrime,4,"(768, 543, 880, 557)"
...,...,...,...
429,Rectangle,12,"(740, 601, 1012, 615)"
430,Rectangle,13,"(740, 619, 788, 633)"
431,Rectangle,15,"(740, 657, 956, 672)"
432,Rectangle,16,"(740, 676, 1100, 691)"


In [9]:
df_line_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber", "FixationStart", "FixationEnd", "LineNumber"])
for snippet in tqdm(snippets):
    df_lines_per_algo = df_lines[df_lines["Algorithm"] == snippet]

    for participant in Participants:
        df_grouped = df_eyetracking_events[(df_eyetracking_events["Algorithm"] == snippet) & (df_eyetracking_events["Participant"] == participant)]

        if len(df_grouped) == 0:
            continue

        eyetracking_path = df_grouped["Path"].values[0]
        df_current_eyetracking = pd.read_csv(eyetracking_path)
        label = df_current_eyetracking["label"].unique()
        df_fix = df_current_eyetracking[df_current_eyetracking["label"] == "FIXA"]
        df_fix = df_fix.reset_index()
        df_fix["y_range"] = (df_fix["end_y"] - df_fix["start_y"]).apply(abs)
        for fix_idx, fix_row in df_fix.iterrows():
            y_fix_low = min(fix_row["start_y"], fix_row["end_y"])
            y_fix_high = max(fix_row["start_y"], fix_row["end_y"])

            was_in_aoi = False
            for _, line_row in df_lines_per_algo.iterrows():
                line_number = line_row["Line"]
                bounding_box = line_row["BoundingBox"]
                y_upper = bounding_box[1]
                y_lower = bounding_box[3]
                # check if any value is true
                if y_upper <= y_fix_low <= y_lower or y_upper <= y_fix_high <= y_lower or y_fix_low < y_upper < y_fix_high or y_fix_low < y_lower < y_fix_high:
                    df_line_fixation_per_participant.loc[len(df_line_fixation_per_participant)] = [snippet, participant, fix_idx, fix_row["start_time"], fix_row["end_time"], line_number]
                    was_in_aoi = True
                    break
            if not was_in_aoi:
                    df_line_fixation_per_participant.loc[len(df_line_fixation_per_participant)] = [snippet, participant, fix_idx, fix_row["start_time"], fix_row["end_time"], None]

df_line_fixation_per_participant

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Participant,FixationNumber,FixationStart,FixationEnd,LineNumber
0,IsPrime,1,0,0.000,0.228,None
1,IsPrime,1,1,0.264,0.440,None
2,IsPrime,1,2,0.656,0.736,None
3,IsPrime,1,3,0.756,0.884,None
4,IsPrime,1,4,1.148,1.348,0
...,...,...,...,...,...,...
172988,Rectangle,71,30,8.812,8.944,3
172989,Rectangle,71,31,8.996,9.080,None
172990,Rectangle,71,32,9.272,9.404,None
172991,Rectangle,71,33,9.436,9.596,2


In [10]:
# save the result
df_line_fixation_per_participant.to_csv("./data/fixation_per_participant_per_line.csv", index=False)

In [19]:
# load the result
df_line_fixation_per_participant = pd.read_csv("./data/fixation_per_participant_per_line.csv")

## Calculate Metrics based on Code Lines

In [13]:
df_line_fixation_per_participant = df_line_fixation_per_participant[df_line_fixation_per_participant["LineNumber"].isna() == False]

In [14]:
# Calculate the LOCs
df_snippet_length = pd.DataFrame(columns=["Algorithm", "LOC"])
for snippet in tqdm(Algorithms):
    aoi_token_generator = f"./../CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    with open(aoi_token_generator) as f:
        aoi_list = json.load(f)
        data = aoi_list["source-code"]
        LOC = len(data)
        df_snippet_length.loc[len(df_snippet_length)] = [snippet, LOC]
df_snippet_length

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,LOC
0,IsPrime,8
1,SiebDesEratosthenes,20
2,IsAnagram,28
3,RemoveDoubleChar,19
4,BinToDecimal,9
5,PermuteString,23
6,Power,7
7,BinarySearch,15
8,ContainsSubstring,21
9,ReverseArray,7


In [15]:
# read in Behavioural and Skills data
df_behavioral = pd.read_csv('./data/filteredData/fixation_stats.csv', sep=";")
df_behavioral = df_behavioral[df_behavioral["IsOutlier"] == False]
df_behavioral = df_behavioral[["Participant", "Algorithm", "Duration", "SkillScore"]]
df_behavioral

,Participant,Algorithm,Duration,SkillScore
0,1,IsPrime,12.390280,0.332799
2,1,IsAnagram,109.615724,0.332799
3,1,RemoveDoubleChar,53.456276,0.332799
4,1,BinToDecimal,49.922091,0.332799
5,1,PermuteString,109.888549,0.332799
...,...,...,...,...
1066,71,GreatestCommonDivisor,30.757360,0.437218
1067,71,DumpSorting,113.368945,0.437218
1068,71,BinomialCoefficient,50.637861,0.437218
1069,71,IsAnagram,110.995754,0.437218


In [19]:
# merge dataframes together to access every possible combination of participant, algorithm and duration,skillscore
df_combined = pd.merge(df_line_fixation_per_participant, df_snippet_length, on=["Algorithm"])
df_combined = pd.merge(df_combined, df_behavioral, on=["Participant", "Algorithm"])

# transform the data to seconds
df_combined["FixationStart"] = df_combined["FixationStart"]
df_combined["FixationEnd"] = df_combined["FixationEnd"]

# Helper function to calculate the coverage of LOC after 'percentage' percent from the whole duration
def loc_coverage_after_time_percentage(df, percentage):
    end_duration = df["Duration"].iloc[0]
    loc = df["LOC"].iloc[0]
    max_duration = end_duration * percentage
    df_filtered = df[df["FixationEnd"] <= max_duration]
    unique_loc = df_filtered["LineNumber"].nunique()
    return unique_loc / loc

# calculate LOC coverage after a certain percentage of the duration
df_10 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.1))
df_20 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.2))
df_30 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.3))
df_40 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.4))
df_50 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.5))
df_60 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.6))
df_70 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.7))
df_80 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.8))
df_90 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.9))
df_100 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 1.))

df_10 = df_10.reset_index()
df_20 = df_20.reset_index()
df_30 = df_30.reset_index()
df_40 = df_40.reset_index()
df_50 = df_50.reset_index()
df_60 = df_60.reset_index()
df_70 = df_70.reset_index()
df_80 = df_80.reset_index()
df_90 = df_90.reset_index()
df_100 = df_100.reset_index()


df_10 = df_10.groupby(["Participant"]).mean().values.reshape(37, )
df_20 = df_20.groupby(["Participant"]).mean().values.reshape(37, )
df_30 = df_30.groupby(["Participant"]).mean().values.reshape(37, )
df_40 = df_40.groupby(["Participant"]).mean().values.reshape(37, )
df_50 = df_50.groupby(["Participant"]).mean().values.reshape(37, )
df_60 = df_60.groupby(["Participant"]).mean().values.reshape(37, )
df_70 = df_70.groupby(["Participant"]).mean().values.reshape(37, )
df_80 = df_80.groupby(["Participant"]).mean().values.reshape(37, )
df_90 = df_90.groupby(["Participant"]).mean().values.reshape(37, )
df_100 = df_100.groupby(["Participant"]).mean().values.reshape(37, )

# extract the raw skillscore per participant
df_skill = df_behavioral[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# create a dataframe with all the LOC coverage and Participant
df_code_coverage = pd.DataFrame({"Participant": participants,
                                 "10%" : df_10,
                                 "20%" : df_20,
                                 "30%" : df_30,
                                 "40%" : df_40,
                                 "50%" : df_50,
                                 "60%" : df_60,
                                 "70%" : df_70,
                                 "80%" : df_80,
                                 "90%" : df_90,
                                 "100%" : df_100})

df_code_coverage.set_index("Participant", inplace=True, drop=True)

# Merge LOC coverage with skillscore with correlations
df_code_coverage = pd.merge(df_code_coverage, df_skill, on=["Participant"])
df_code_coverage.set_index("Participant", inplace=True, drop=True)
df_code_coverage

,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,SkillScore
Participant,,,,,,,,,,,
1,0.146680,0.285978,0.383355,0.465116,0.532492,0.597804,0.633861,0.665345,0.720942,0.766538,0.332799
2,0.149559,0.246279,0.350841,0.418662,0.469966,0.520825,0.584122,0.627649,0.670700,0.706364,0.381621
3,0.300844,0.486081,0.585241,0.682984,0.741255,0.772132,0.801960,0.817506,0.827659,0.843368,0.315012
4,0.125477,0.293682,0.411732,0.500067,0.549466,0.588629,0.629069,0.658221,0.697029,0.762320,0.426317
5,0.184094,0.273982,0.352204,0.421866,0.492056,0.573894,0.652752,0.686290,0.725607,0.768767,0.313899
6,0.109909,0.239896,0.376225,0.487325,0.593846,0.644880,0.703852,0.762052,0.803980,0.826108,0.318673
7,0.084013,0.209871,0.301367,0.372163,0.427588,0.452712,0.478599,0.506967,0.565844,0.601748,0.408083
10,0.091872,0.176927,0.267090,0.335192,0.429375,0.495449,0.536660,0.594485,0.628502,0.668651,0.350811
11,0.119474,0.271348,0.390908,0.408926,0.467726,0.506358,0.527032,0.543912,0.586293,0.716151,0.165306


In [21]:
# describe the LOC coverage
df_code_coverage.describe()

,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,SkillScore
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,0.120873,0.234169,0.327526,0.404604,0.473058,0.524575,0.571188,0.611315,0.650219,0.705822,0.325261
std,0.080188,0.109248,0.122559,0.132468,0.128267,0.125982,0.122727,0.122042,0.117265,0.101936,0.102608
min,0.008798,0.073996,0.142943,0.215073,0.258841,0.293447,0.330956,0.352365,0.380415,0.455626,0.144164
25%,0.059455,0.141294,0.242361,0.295407,0.376197,0.428237,0.480799,0.517338,0.567690,0.653487,0.262502
50%,0.110239,0.239896,0.343544,0.408926,0.469966,0.520825,0.584122,0.627649,0.670700,0.723539,0.317626
75%,0.175967,0.291381,0.390908,0.479837,0.549466,0.599201,0.634092,0.680632,0.720942,0.768767,0.381621
max,0.351280,0.549756,0.667279,0.777498,0.837492,0.856962,0.876683,0.896048,0.899556,0.904001,0.659556


In [22]:
# spearman correlation for LOC coverage and skillscore
df_code_coverage.corrwith(df_code_coverage["SkillScore"])

10%          -0.230927
20%          -0.256916
30%          -0.273375
40%          -0.218091
50%          -0.251149
60%          -0.255258
70%          -0.246854
80%          -0.238073
90%          -0.221308
100%         -0.272816
SkillScore    1.000000
dtype: float64